In [1]:
# Instalação dos pacotes necessários
!pip install pandas



[notice] A new release of pip is available: 23.2.1 -> 25.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [2]:
import pandas as pd
from collections import deque, defaultdict
import math

# --- 1) Carrega o banco de funcionários e calcula capacidade por setor (horas/dia) ---
employees_df = pd.read_csv('Funcionarios.csv')

# Extrai todos os setores que aparecem na coluna 'Setores'
sectors = employees_df['Setores'].unique().tolist()

# Calcula a capacidade (soma de horas por dia) de cada setor
capacity = {}
for s in sectors:
    mask = (employees_df['Setores'] == s)
    capacity[s] = employees_df.loc[mask, 'HorasPorDia'].sum()
    if capacity[s] == 0:
        raise ValueError(f"Setor {s} não tem funcionários alocados!")

# --- 2) Define o escopo original: horas necessárias por tarefa e setor ---
# Ajuste os valores de horas conforme seu projeto
task_hours = {
    "T1": {"Level Design": 560, "Iluminação": 160, "Modelador 3D": 120, "Programador": 220,  "QA": 40},
    

}

# --- 3) Expande cada Tarefa em fases por setor, convertendo para dias e estabelecendo dependências ---
tasks = {}
for T, hours_dict in task_hours.items():
    for S, h in hours_dict.items():
        key = f"{T}_{S}"
        dur = h / capacity[S]   # duração em dias (float)
        # Definição de predecessores (dependências) por setor:
        if S in ("Modelador 3D", "Level Design"):
            preds = []
        elif S == "Iluminação":
            preds = [f"{T}_Level Design"]
        elif S == "Programador":
            preds = [f"{T}_Level Design"]
        elif S == "QA":
            preds = [
                f"{T}_Level Design", 
                f"{T}_Modelador 3D", 
                f"{T}_Iluminação", 
                f"{T}_Programador"
            ]
        else:
            preds = []
        tasks[key] = {
            "duration": dur,
            "predecessors": preds
        }

# --- 4) Monta grafo de sucessores e grau de entrada para ordenação topológica ---
successors = defaultdict(list)
in_degree  = { t: 0 for t in tasks }
for t, info in tasks.items():
    for p in info["predecessors"]:
        successors[p].append(t)
        in_degree[t] += 1

# --- 5) Forward Pass: calcula ES e EF ---
ES, EF = {}, {}
queue = deque([t for t, deg in in_degree.items() if deg == 0])
while queue:
    t = queue.popleft()
    es = max((EF[p] for p in tasks[t]["predecessors"]), default=0)
    ES[t] = es
    EF[t] = es + tasks[t]["duration"]
    for succ in successors[t]:
        in_degree[succ] -= 1
        if in_degree[succ] == 0:
            queue.append(succ)
project_duration = max(EF.values())

# --- 6) Backward Pass: calcula LS e LF ---
LF = {t: project_duration for t in tasks}
LS = {}
for t in sorted(EF, key=lambda x: EF[x], reverse=True):
    lf = min((LS[s] for s in successors[t]), default=project_duration)
    LF[t] = lf
    LS[t] = lf - tasks[t]["duration"]

# --- 7) Identifica o caminho crítico (folga zero) ---
# Considera como crítico qualquer fase com folga <= 1e-6
critical_path = [t for t in sorted(tasks, key=lambda x: ES[x])
                 if LS[t] - ES[t] <= 1e-6]

# --- 8) Impressão resumida ---
print(f"Duração total do projeto (dias): {project_duration:.2f}\n")
print(f"{'Fase':<25} ES     EF     LS     LF    Folga")
for t in sorted(tasks, key=lambda x: ES[x]):
    folga = LS[t] - ES[t]
    print(f"{t:<25} {ES[t]:6.2f} {EF[t]:6.2f} {LS[t]:6.2f} {LF[t]:6.2f} {folga:6.2f}")

print("\nCaminho crítico:")
print(" → ".join(critical_path))


Duração total do projeto (dias): 11.88

Fase                      ES     EF     LS     LF    Folga
T1_Level Design             0.00   5.09  -0.00   5.09  -0.00
T1_Modelador 3D             0.00   3.53   5.49   9.02   5.49
T1_Iluminação               5.09   7.31   6.80   9.02   1.71
T1_Programador              5.09   9.02   5.09   9.02  -0.00
T1_QA                       9.02  11.88   9.02  11.88   0.00

Caminho crítico:
T1_Level Design → T1_Programador → T1_QA
